# 4th lab:

***prepared by Pavel 3P & Markov Artur ***


### Area info:

In [209]:
bottom_left = (29.073321247506765, 49.845775018245774)
upper_right = (31.986007792928522, 51.278667808079206)

#Filtering options:
from_time = "2019-04-01T00:00:00Z"
to_time = "2020-05-01T00:00:00Z"
maxCloudCoverage = 0

#In px:
width = 2000
height = 2000

## First part:

### Create session with server:

In [243]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session


client_id = "8d7432b3-19c9-4a6e-9d02-fcf865d1a6c6"
client_secret = "fdR{0FFu(}F)KD/s@>9(w;28*]MHkQJoo_I!*Q73"


client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)


token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

### get_band(band_num: int, img_format: str ="tif"):
___

Takes as arguments band number and image format to save. It downloads and saves 1-band image with target band to *./data/{band name}.{img_format}*
___

In [240]:
import requests
import json
import os

def get_band(band_num: int, img_format: str = "tif"):
    band_names = {
        2: "b",
        3: "g",
        4: "r",
        8: "nir",
    }
    req_data = {
        "input": {
            "bounds": {
                "bbox": [
                    bottom_left[0],
                    bottom_left[1],
                    upper_right[0],
                    upper_right[1],
                ]
            },
            "data": [
                {
                    "type": "S2L2A",
                    "dataFilter": {
                        "timeRange": {
                            "from": from_time,
                            "to": to_time,
                        },
                        "maxCloudCoverage": maxCloudCoverage,
                    },
                }
            ]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": [{
                "format": {
                    "type": "image/tiff"
                }
            }]
        },
        "evalscript": """
        //VERSION=3

        function setup() {
          return {
            input: ["B0""" + str(band_num) + """"], 
            output: {
              bands: 1
            }
          };
        }

        function evaluatePixel(
          sample,
          scenes,
          inputMetadata,
          customData,
          outputMetadata
        ) {
          return [2.5 * sample.B0""" + str(band_num) + """]; 
        }
        """
    }
    # 0 * sample.B04, 0 * sample.B03, 2.5 * sample.B02, 4-3-2-8 #"B03", "B04", "B08"
    headers = {
        "Authorization": f"Bearer {token['access_token']}",
    }
    url = "https://services.sentinel-hub.com/api/v1/process"

    response = requests.post(url, json=req_data, headers=headers)
    
    if not os.path.isdir("data"):
        os.mkdir("data")
    
    with open(f"data/{band_names[band_num]}.{img_format}", "wb") as img:
        img.write(response.content)

### Getting all necessary bands:

In [244]:
get_band(2)
get_band(3)
get_band(4)
get_band(8)